In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import time
import collections
import random
import pandas as pd
import datetime
import scipy as sci
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import torch



In [ ]:

##Reference  https://www.marksmath.org/classes/Spring2018NumericalAnalysis/code/nBody.html


r0=2
v0=2


########clumps r4, ring r2, double mill,r2, flock r4, single mill r4 ###############
lambda1 = [0, -1.076E-01, -2.024E-02, -1.172E+00, 1.999E-01, -5.406E-01]
lambda2 = [0, -4.643E-05, -4.010E-02, 1.804E+00, -1.345E-01, 8.589E-01]
lambda3 = [0, 8.328E-01, 1.215E-03, -7.604E-01, -1.015E-01, 5.433E-01]
lambda4 = [0, -6.884E-01, 3.202E-02, 3.946E-01, 1.146E-01, 5.192E-01]

power1 = [0, 2.932E-01, -1.726E-01, -1.186E-01, -9.288E-02, 7.426E-01]
power2 = [0, -4.218E-02, 7.546E-01, -3.836E-01, 1.826E-01, 3.296E-02]
power3 = [0, -4.846E-04, -3.115E-01, -1.086E-01, 1.322E-01, -3.231E-02]


########## velocity coeff #########
lambda1_v = [0, 0, 0, 0, -1.026E-01, 0]
lambda2_v = [0, 0, 0, 0, -3.282E-01, 0]
lambda3_v = [0, 0, 0, 0, -1.597E-01, 0]
lambda4_v = [0, 0, 0, 0, 4.405E-01, 0]

power1_v = [0, 0, 0, 0, -2.314E-01, 0]
power2_v = [0, 0, 0, 0, -3.216E-01, 0]
power3_v = [0, 0, 0, 0, -2.862E-01, 0]


def total_infer_x(t, s,i):
    n_array = np.array(n)
    self_index=np.repeat(i,n-1)
    
    others_index=list(range(n))
    others_index.remove(i)
    
    
    x1=np.take(s, self_index)
    y1=np.take(s, self_index + n_array)
    x2=np.take(s, others_index)
    y2=np.take(s, others_index + n_array)
    dist=((x1-x2)**2 + (y1-y2)**2)**(1/2)
    

    vi_x=np.take(s, 2*n_array + self_index) #self
    vi_y=np.take(s, 3*n_array + self_index) #self
    vj_x=np.take(s, 2*n_array + others_index)
    vj_y=np.take(s, 3*n_array + others_index)
    
    vel_rel=((vi_x-vj_x)**2 + (vi_y-vj_y)**2)**(1/2)
    local_dist_func = np.heaviside(int_range - dist,1)
    
    decay = np.exp(-t)
    
    a1 = lambda1[k]*(1-decay) + lambda1[k-1]*decay
    a2 = lambda2[k]*(1-decay) + lambda2[k-1]*decay
    a3 = lambda3[k]*(1-decay) + lambda3[k-1]*decay
    a4 = lambda4[k]*(1-decay) + lambda4[k-1]*decay
    A1 = power1[k]*(1-decay) + power1[k-1]*decay
    A2 = power2[k]*(1-decay) + power2[k-1]*decay
    A3 = power3[k]*(1-decay) + power3[k-1]*decay
    
    b1 = lambda1_v[k]*(1-decay) + lambda1_v[k-1]*decay
    b2 = lambda2_v[k]*(1-decay) + lambda2_v[k-1]*decay
    b3 = lambda3_v[k]*(1-decay) + lambda3_v[k-1]*decay
    b4 = lambda4_v[k]*(1-decay) + lambda4_v[k-1]*decay
    B1 = power1_v[k]*(1-decay) + power1_v[k-1]*decay
    B2 = power2_v[k]*(1-decay) + power2_v[k-1]*decay
    B3 = power3_v[k]*(1-decay) + power3_v[k-1]*decay
    
    
    F_int_x=(x1-x2)/dist*(a1*(dist**A1)+a2*(dist**A2) + a3*(dist**A3)+a4) + \
    (vi_x-vj_x)/vel_rel*(b1*(dist**B1)+b2*(dist**B2) \
                         + b3*(dist**B3) + b4)*local_dist_func

    
    total_interaction_x=sum(F_int_x)
    
    v_i_x=s[2*n+i]
    v_i_y=s[3*n+i]
    v_i_mag = (v_i_x**2+v_i_y**2)**(1/2)
    self_propel_x = 1*(v_i_x/v_i_mag)*(v0-v_i_mag)

    return self_propel_x+total_interaction_x

def total_infer_y(t, s,i):
    n_array = np.array(n)
    self_index=np.repeat(i,n-1)
    
    others_index=list(range(n))
    others_index.remove(i)
    
    x1=np.take(s, self_index)
    y1=np.take(s, self_index + n_array)
    x2=np.take(s, others_index)
    y2=np.take(s, others_index + n_array)
    dist=((x1-x2)**2 + (y1-y2)**2)**(1/2)
    

    vi_x=np.take(s, 2*n_array + self_index) #self
    vi_y=np.take(s, 3*n_array + self_index) #self
    vj_x=np.take(s, 2*n_array + others_index)
    vj_y=np.take(s, 3*n_array + others_index)
    
    vel_rel=((vi_x-vj_x)**2 + (vi_y-vj_y)**2)**(1/2)
    local_dist_func = np.heaviside(int_range - dist,1)
    
    decay = np.exp(-t)
    
    a1 = lambda1[k]*(1-decay) + lambda1[k-1]*decay
    a2 = lambda2[k]*(1-decay) + lambda2[k-1]*decay
    a3 = lambda3[k]*(1-decay) + lambda3[k-1]*decay
    a4 = lambda4[k]*(1-decay) + lambda4[k-1]*decay
    A1 = power1[k]*(1-decay) + power1[k-1]*decay
    A2 = power2[k]*(1-decay) + power2[k-1]*decay
    A3 = power3[k]*(1-decay) + power3[k-1]*decay
    
    b1 = lambda1_v[k]*(1-decay) + lambda1_v[k-1]*decay
    b2 = lambda2_v[k]*(1-decay) + lambda2_v[k-1]*decay
    b3 = lambda3_v[k]*(1-decay) + lambda3_v[k-1]*decay
    b4 = lambda4_v[k]*(1-decay) + lambda4_v[k-1]*decay
    B1 = power1_v[k]*(1-decay) + power1_v[k-1]*decay
    B2 = power2_v[k]*(1-decay) + power2_v[k-1]*decay
    B3 = power3_v[k]*(1-decay) + power3_v[k-1]*decay
    
    force_y = (y1-y2)/dist*(a1*(dist**A1)+a2*(dist**A2) + a3*(dist**A3)+a4) + \
    (vi_y-vj_y)/vel_rel*(b1*(dist**B1)+b2*(dist**B2) \
                         + b3*(dist**B3) + b4)*local_dist_func
    
    F_int_y=(y1-y2)/dist*(a1*(dist**A1)+a2*(dist**A2) + a3*(dist**A3)+a4) + \
    (vi_y-vj_y)/vel_rel*(b1*(dist**B1)+b2*(dist**B2) \
                         + b3*(dist**B3) + b4)*local_dist_func

    
    total_interaction_y=sum(F_int_y)
    
    v_i_x=s[2*n+i]
    v_i_y=s[3*n+i]
    v_i_mag = (v_i_x**2+v_i_y**2)**(1/2)
    self_propel_y = 1*(v_i_y/v_i_mag)*(v0-v_i_mag)

    return self_propel_y+total_interaction_y


def F_infer_ivp(t,s):
    return np.concatenate((s[2*n:3*n],s[3*n:4*n],[total_infer_x(t,s,i) for i in range(n)], [total_infer_y(t,s,i) for i in range(n)]))

def F_infer_v_ivp(t,s):
    return np.concatenate((s[2*n:3*n],s[3*n:4*n],\
                           [total_infer_x(t,s,i) for i in range(n)]+np.random.normal(0,10,size=n), \
                           [total_infer_y(t,s,i) for i in range(n)]+np.random.normal(0,10,size=n)))
def F_noise(t,s):
    return np.concatenate((s[2*n:3*n],s[3*n:4*n], np.random.normal(0,10,size=n), np.random.normal(0,10,size=n)))

In [ ]:
##########Inital condition ###########

position_list_x=[]
position_list_y=[]
velocity_list_x=[]
velocity_list_y=[]
random.seed(101)
# random.seed()
for i in range(40):
    position_list_x.append(random.uniform(-1,1))
    position_list_y.append(random.uniform(-1,1))
    velocity_list_x.append(random.uniform(-1,1))
    velocity_list_y.append(random.uniform(-1,1))



eps = torch.finfo(torch.float32).eps

nodes_G = torch.tensor([position_list_x, position_list_y, velocity_list_x, velocity_list_y], dtype=torch.float32)
nodes_np=nodes_G.to('cpu').detach().numpy()

s0=nodes_np.reshape(-1)
n=nodes_np.shape[1]

In [ ]:
dt=0.1
T_0=30
t_0=np.arange(0,T_0+dt,dt)
print(t_0.shape)

# T_1=30
T_1 = 30
t_1=np.arange(0,T_1+dt,dt)
print(t_1.shape)

T_2=30
t_2=np.arange(0,T_2+dt,dt)
print(t_2.shape)

T_3=50
t_3=np.arange(0,T_3+dt,dt)
print(t_3.shape)

T_noise = 5
t_noise=np.arange(0,T_noise+dt,dt)
print(t_noise.shape)

T_4=100
t_4=np.arange(0,T_4+dt,dt)
print(t_4.shape)



In [ ]:
tik = time.time()
now = datetime.datetime.now()
print(now)

k= 1
int_range =0 
solution=solve_ivp(F_infer_ivp,[0,T_0],s0, method='RK45', max_step=0.01, t_eval=t_0)
print(solution)
print('Step {:01d} end'.format(k))

k= 2
int_range = 0
solution_1=solve_ivp(F_infer_ivp,[0,T_1],solution.y[:,-1], method='RK45', max_step=0.01, t_eval=t_1)
print(solution_1)
print('Step {:01d} end'.format(k))


k= 3
int_range =0
solution_2 = solve_ivp(F_infer_ivp,[0,T_2], solution_1.y[:,-1], method='RK45', max_step=0.01, t_eval=t_2)
print(solution_2)
print('Step {:01d} end'.format(k))


k = 4
v0=2
int_range =2
solution_3 = solve_ivp(F_infer_v_ivp,[0,T_3], solution_2.y[:,-1], method='RK45', max_step=0.01, t_eval=t_3)
print(solution_3)
print('Step {:01d} end'.format(k))

### noise step ###
int_range =0
v0=0
solution_noise = solve_ivp(F_noise,[0,T_noise], solution_3.y[:,-1], method='RK45', max_step=0.01, t_eval=t_noise)
print(solution_noise)
print('Step {:01d} end'.format(k))

k = 5
v0=2
int_range =0
solution_4 = solve_ivp(F_infer_ivp,[0,T_4], solution_noise.y[:,-1], method='RK45', max_step=0.01, t_eval=t_4)
print(solution_4)
print('Step {:01d} end'.format(k))

print('time: {:.4f}s'.format(time.time() - tik))

In [ ]:
import matplotlib.animation as animation
speed = 2

plt.rc('font', size=20, family = 'Arial')
t_plot=np.arange(0,T_0 + T_1 + T_noise + T_2 + T_3 + T_4 + dt*speed ,dt*speed)

solution_plot=np.transpose(solution.y)
solution_plot_1=np.transpose(solution_1.y)
solution_plot_noise = np.transpose(solution_noise.y)
solution_plot_2=np.transpose(solution_2.y)
solution_plot_3=np.transpose(solution_3.y)
solution_plot_4=np.transpose(solution_4.y)

h = np.concatenate((solution_plot[:,0:2*n], \
                    solution_plot_1[1:,0:2*n], \
                    solution_plot_2[1:,0:2*n], \
                    solution_plot_3[1:,0:2*n], \
                    solution_plot_noise[1:,0:2*n], \
                    solution_plot_4[1:,0:2*n]), axis=0)
velocity = np.concatenate((solution_plot[:,2*n:4*n], \
                           solution_plot_1[1:,2*n:4*n], \
                           solution_plot_2[1:,2*n:4*n], \
                           solution_plot_3[1:,2*n:4*n], \
                           solution_plot_noise[1:,2*n:4*n], \
                           solution_plot_4[1:,2*n:4*n]), axis=0)


fig = plt.figure(figsize=(8, 8))
ax = plt.axes()
tail_length = 10


############################### order parameter #####################
velocity_x = velocity[:,0:n]
velocity_y = velocity[:,n:]
center_x = np.mean(h[:,0:n],axis=1,keepdims=True)
center_y = np.mean(h[:,n:],axis=1,keepdims=True)
r_x, r_y = h[:,0:n]-center_x, h[:,n:]-center_y

r_mag = np.sqrt(np.square(r_x)+np.square(r_y))
vel_mag=np.sqrt(np.square(velocity_x)+np.square(velocity_y)) #N X V
vel_mag_sum=np.sum(vel_mag, axis=1)

vel_x_sum = np.sum(velocity_x, axis=1)
vel_y_sum = np.sum(velocity_y, axis=1)

vel_vec_mag = np.sqrt(np.square(vel_x_sum)+np.square(vel_y_sum)) # abs(sum of velocity vector)

order_para = vel_vec_mag/vel_mag_sum

ang_mom = r_x*velocity_y - r_y * velocity_x

denom = np.sum(r_mag*vel_mag, axis=1)
ang_mom_sum = np.abs(np.sum(r_x*velocity_y - r_y * velocity_x, axis=1))
ang_mom_abs = np.sum(np.abs(r_x*velocity_y - r_y * velocity_x), axis=1)

order_para_rot = ang_mom_sum/denom
order_para_rot_abs = ang_mom_abs/denom

##########################################################################
init_color = np.array([0.,0.,0.])
cw_color =  np.array([0.,100.,250.])
ccw_color =  np.array([240.,84.,0.])
flock_color = np.array([145.,84.,192.])
agent_color = np.array([105./255,105./255,105./255,0.8])


window = 10

def animate(i):
    ax.clear()
    ax.scatter(h[i*speed,0:n],h[i*speed,n:], color=agent_color, s=200, zorder=0)
    for j in range(n):
        ax.plot(h[i*speed - tail_length:i*speed+1,j],h[i*speed- tail_length:i*speed+1,n+j], '--', c=agent_color, lw=1, ms=1)
    
    rot_color = []
    if t_plot[i]<T_0+ T_1 + T_noise+T_2+T_3 and t_plot[i]>T_0+T_1 + T_2:
        for ii in range(len(ang_mom[i*speed,:])):
            if order_para[i*speed]>0.7:
                rot_color.append(np.append(flock_color*(order_para[i*speed]-0.7)/0.3/255, [0.8]))
            else:
                rot_color.append(np.append(init_color,[0.8]))
    else:
        for ii in range(len(ang_mom[i*speed,:])):
            if ang_mom[i*speed,ii]>0 and order_para_rot_abs[i*speed]>0.7:
                rot_color.append(np.append(ccw_color*(order_para_rot_abs[i*speed]-0.7)/0.3/255, [0.8]))
            elif ang_mom[i*speed,ii]<0 and order_para_rot_abs[i*speed]>0.7:
                rot_color.append(np.append(cw_color*(order_para_rot_abs[i*speed]-0.7)/0.3/255, [0.8]))
            else:
                rot_color.append(np.append(init_color,[0.8]))
    
    
    origin=np.array([h[i*speed,0:n], h[i*speed,n:]])
    ax.quiver(*origin, velocity[i*speed,0:n], velocity[i*speed,n:], scale_units='xy', angles='xy', color=rot_color, \
              scale=1.5, width=0.006, headwidth=5,zorder=2)

    center_x_i = np.mean(h[i*speed,0:n])
    center_y_i = np.mean(h[i*speed,n:])
    
    if t_plot[i]<T_0:
        ax.set_title('Time: {:.2f}\n Step 1, Clumps'.format(t_plot[i]))
    elif t_plot[i]<T_0+T_1:
        ax.set_title('Time: {:.2f}\n Step 2, Ring'.format(t_plot[i]))
    elif t_plot[i]<T_0+T_1 + T_2:
        ax.set_title('Time: {:.2f}\n Step 3, Double mill'.format(t_plot[i]))
    elif t_plot[i]<T_0+ T_1 +T_2 + T_3:
        ax.set_title('Time: {:.2f}\n Step 4, Flock'.format(t_plot[i]))
    elif t_plot[i]<T_0+ T_1 + T_noise+T_2+T_3:
        ax.set_title('Time: {:.2f}\n Noise only'.format(t_plot[i]))
    else:
        ax.set_title('Time: {:.2f}\n Step 5, Single mill'.format(t_plot[i]))
        
    ax.set_xticks(np.arange(-100, 100, 5))
    ax.set_yticks(np.arange(-100, 100, 5))
    ax.set_xlim(center_x_i - window, center_x_i +window)
    ax.set_ylim(center_y_i - window, center_y_i +window)



anim = animation.FuncAnimation(fig, animate, frames=len(t_plot))


from IPython.display import HTML
HTML(anim.to_html5_video())